# Predict With a Saved ML Model

This notebook loads a previously saved ML model (STAC item) and applies it to a data cube.

In [ ]:
import openeo # type: ignore

In [ ]:
connection = openeo.connect(url="http://127.0.0.1:8000")
connection.authenticate_basic("brian", "123456")

## Load the saved model


In [ ]:
model = connection.load_ml_model("tempcnn_rondonia")

## Load Sentinel-2 data and predict

In [ ]:
datacube = connection.load_collection(
    collection_id="mpc-sentinel-2-l2a",
    spatial_extent={"west": -63.6078, "south": -8.95630, "east": -63.25790, "north": -8.72290},
    temporal_extent=["2022-01-01", "2022-12-31"],
    bands=["b02", "b03", "b04", "b08", "cloud"]
)

In [ ]:
datacube = datacube.process(
    process_id="cube_regularize",
    arguments={
        "data": datacube,
        "period": "P16D",
        "resolution": 600
    }
)

In [ ]:
prediction = model.predict(datacube)

## Save results and run the job

In [ ]:
result = prediction.save_result(format="GeoTiff")
job = result.create_job(
    title="Predict using saved model",
    description="Apply saved ML model to a Sentinel-2 cube"
)
job.start_and_wait()
results = job.get_results()

In [ ]:
results.download_files("data/outputs/")